# Setup

In [33]:
import sqlite3
import pandas as pd
import datetime
import wordcloud as wc
from wordcloud import WordCloud, STOPWORDS
import sys
import numpy as np

# Grab text messages

### Grab your iMessages courtesy of https://stmorse.github.io/journal/iMessage.html 

In [5]:
# path to message db
conn = sqlite3.connect('/Users/wbr/Library/Messages/chat.db')
c = conn.cursor()

In [6]:
# grab the messages from chat id 194, which is my the id for my Family group chat
cmd1 = 'SELECT ROWID, text, handle_id, \
            datetime(date + strftime(\'%s\',\'2001-01-01\'), \'unixepoch\') as date_utc \
            FROM message T1 \
            INNER JOIN chat_message_join T2 \
                ON T2.chat_id=194 \
                AND T1.ROWID=T2.message_id \
            ORDER BY T1.date'
c.execute(cmd1)
df_msg = pd.DataFrame(c.fetchall(), columns=['id', 'text', 'sender', 'time'])

In [10]:
df_msg.head()

,id,text,sender,time
0,17605,I'll be on St. Louis on the air tomorrow from ...,8,2015-12-02 18:13:10
1,17606,Nice,1,2015-12-02 18:13:32
2,17607,Tomorrow is my birthday,3,2015-12-02 18:13:52
3,17608,I know. I'll see if I can give you a shout out.,8,2015-12-02 18:14:38
4,17609,Hahaha,3,2015-12-02 18:14:50


# Make a wordcloud 

### wordcloud source https://github.com/nikhilkumarsingh/wordcloud-example/blob/master/mywc.py

In [30]:
# get path to directory
currdir = '/Users/WBR/walter/python_psc290/hw'


In [55]:
cat_words = df_msg.text.str.cat(sep=',')


In [60]:
tuples = tuple([tuple(x) for x in df_msg.text.value_counts().reset_index().values])
print (tuples)

(('￼', 267), ('Loved an image', 106), ('Loved a movie', 60), ('Liked an image', 44), ('Haha', 42), ('Nice', 32), ('Oh', 18), ('Lol', 17), ('Ha', 14), ('K', 12), ('￼I liked this Word of the day. Can you guess what is it?\xa0\xa0/via @wordwordapp', 11), ('http://en.wordwordapp.com', 11), ('Yeah', 11), ('Ok', 10), ('lol', 9), ('?', 9), ('Cute', 8), ('￼￼', 8), ('Fun', 8), ('Nice!', 8), ('Yes', 6), ('Wow', 6), ('Cool', 6), ('Liked a movie', 6), ('Hahaha', 5), ('hahahaha', 5), ('Funny', 5), ('Yikes', 5), ('Awww', 5), ('Omg', 5), ('Nice!!', 5), ('Emphasized an image', 4), ('Sweet', 4), ('Fun!!', 4), ('Right', 4), ('What?', 4), ('Yum', 4), ('Ouch', 4), ('Hmmm', 4), ('Ugh', 4), ('No', 4), ('Pretty', 4), ('Yup', 4), ('Awesome!!', 4), ('Jeez', 4), ('yeah', 4), ('Sheesh', 3), ('❤️', 3), ('Ya', 3), ('🙌', 3), ('Wtf', 3), ('Wow!!', 3), ('ha', 3), ('nice', 3), ('???', 3), ('Awesome', 3), ('cool', 3), ('Dang', 3), ('Yep', 3), ('Nope', 3), ('Awwww', 3), ('♥️', 3), ('😍', 3), ('Woah', 3), ('👏🏽👏🏽👏🏽', 3), (

In [63]:
# def get_wiki(query):
#     # get best matching title for given query
#     title = wikipedia.search(query)[0]

#     # get wikipedia page for selected title
#     page = wikipedia.page(title)
#     return page.content


def create_wordcloud(text):
    # create numpy araay for wordcloud mask image
    mask = np.array(Image.open(path.join(currdir, "cloud.png")))

    # create set of stopwords	
    stopwords = set(STOPWORDS)

    # create wordcloud object
    wc = WordCloud(background_color="white",
                    max_words=200, 
                    mask=mask,
                    stopwords=stopwords)

    # generate wordcloud
#     wc.generate(text)
    WordCloud.generate_from_frequencies(text)
    # save wordcloud
    wc.to_file(path.join(currdir, "wc.png"))

In [64]:
create_wordcloud(tuples)

TypeError: generate_from_frequencies() missing 1 required positional argument: 'frequencies'

In [43]:
df_msg.text

0       I'll be on St. Louis on the air tomorrow from ...
1                                                    Nice
2                                 Tomorrow is my birthday
3        I know. I'll see if I can give you a shout out. 
4                                                  Hahaha
5                                                       ￼
6                                                   Ouch!
7                                                       ￼
8                                                      👌🏻
9                                                   Good 
10                                    Trump and newt\n🐵🙈🙉
11                                                     🏆🤔
12                                                      ￼
13                                                  Which
14                                                  Whoah
15                                                   😍🌈🎨🍺
16                                    You painted that?  
17            